In [1]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import pandas as pd
import numpy as np
from pathlib import Path

# Functions

In [2]:
def weighted_mean(x, metric: str):
    if len(x[metric].dropna()) == 0:
        res = 0
    else:
        res = (x[metric] * x["sample_weight"]).sum()/x["sample_weight"].sum()
    col_name = f"{metric} wt_mean"
    return pd.Series({col_name: res})

def weighted_percentile(x, metric: str, percents: int):
    col_name = f"{metric} wt_p{percents}"
    res = _weighted_percentile(x[metric], percents, weights=x["sample_weight"])
    return pd.Series({col_name: res})

def _weighted_percentile(data, percents: int, weights=None):
    """
    perc : percentile in 0 - 1
    """
    # dropna
    idx = ~np.isnan(data) & ~np.isnan(weights)
    data = np.array(data)[idx]
    if len(data) == 0:
        return np.nan
    if weights is None:
        return np.percentile(data, percents)
    weights = np.array(weights)[idx]
    ix = np.argsort(data)
    data = data[ix] # sort data
    weights = weights[ix] # sort weights
    cdf = (np.cumsum(weights) - 0.5 * weights) / np.sum(weights) # 'like' a CDF function
    return np.interp(percents, cdf, data)

def get_summary(df, metric, lower_percentile=0.10, middle_percentile1=0.25, middle_percentile2=0.75, upper_percentile=0.90):
    dfgb = df.groupby(["upgrade_name"])
    dfs = pd.concat([
        dfgb["sample_weight"].count().rename("n_samples"),
        dfgb["sample_weight"].sum().rename("n_represented"),
        dfgb.apply(lambda x: weighted_mean(x, metric)),
        dfgb.apply(lambda x: weighted_percentile(x, metric, percents=lower_percentile)),
        dfgb.apply(lambda x: weighted_percentile(x, metric, percents=middle_percentile1)),
        dfgb.apply(lambda x: weighted_percentile(x, metric, percents=middle_percentile2)),
        dfgb.apply(lambda x: weighted_percentile(x, metric, percents=upper_percentile)),
    ], axis=1)
    dfs = dfs.sort_values(by=f"{metric} wt_mean", ascending=False)

    return dfs

## Whole Community Upgrade Cost Per Dwelling Unit

In [3]:
def community_mean_upgrade_cost(commnity_results):
    results = commnity_results[['upgrade_name',
                       'upgrade_cost_usd',
                                'sample_weight']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results['upgrade_cost_usd'] = results['upgrade_cost_usd']*results['sample_weight']
    
    agg_functions = {'upgrade_name': 'first',
                     'upgrade_cost_usd': 'sum',
                    'sample_weight': 'sum'}
    results = results.groupby(results['upgrade_name']).aggregate(agg_functions)
    results['upgrade_cost_usd'] = results['upgrade_cost_usd']/results['sample_weight']
    results = results.sort_values('upgrade_cost_usd')
    results['upgrade_cost_usd'] = results['upgrade_cost_usd'].round(1) 
    
    print('upgrade_cost_usd per dwelling unit')
    display(results['upgrade_cost_usd'])
    return results['upgrade_cost_usd']

## Whole Community Total Annual Savings

In [4]:
# flavor 1 - with electricity and non-electricity split
def figure_total_savings_1(commnity_results):
    results = commnity_results[['upgrade_name',
                       'saving_energy.electricity_kwh',
                      'saving_energy.natural_gas_therm',
                                'saving_energy.fuel_oil_mmbtu',
                                'saving_energy.propane_mmbtu',
                       'saving_cbill.total_usd',
                       'saving_emission.total_kgCO2e',
                                'sample_weight']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'] = results['saving_energy.natural_gas_therm']*0.1 + results['saving_energy.fuel_oil_mmbtu']+ results['saving_energy.propane_mmbtu']
    results['saving_energy.electricity_kwh'] = results['saving_energy.electricity_kwh']*results['sample_weight']
    results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'] = results['saving_energy.natural_gas_fuel_oil_propane_mmbtu']*results['sample_weight']
    results['saving_cbill.total_usd'] = results['saving_cbill.total_usd']*results['sample_weight']
    results['saving_emission.total_kgCO2e'] = results['saving_emission.total_kgCO2e']*results['sample_weight']
    
    agg_functions = {'upgrade_name': 'first',
                     'saving_energy.electricity_kwh': 'sum',
                     'saving_energy.natural_gas_fuel_oil_propane_mmbtu': 'sum', 
                     'saving_cbill.total_usd': 'sum',
                     'saving_emission.total_kgCO2e': 'sum'}
    results = results.groupby(results['upgrade_name']).aggregate(agg_functions)
    results = results.sort_values('saving_cbill.total_usd')
    
    fig = make_subplots(rows=1, cols=5, horizontal_spacing=0.12, column_widths=[0.001, 1, 1, 1, 1])
    
    fig.add_trace(go.Bar(
        x=[0] * 176,
        y=results['upgrade_name'],
        text=None,
        orientation='h'), row=1, col=1)
    
    fig.add_trace(go.Bar(
        x=results['saving_cbill.total_usd'],
        y=results['upgrade_name'],
        text=results['saving_cbill.total_usd'],
        orientation='h'), row=1, col=2)
    
    fig.add_trace(go.Bar(
        x=results['saving_emission.total_kgCO2e'],
        y=results['upgrade_name'],
        text=results['saving_emission.total_kgCO2e'],
        orientation='h'),row=1, col=3)
    
    fig.add_trace(go.Bar(
        x=results['saving_energy.electricity_kwh'],
        y=results['upgrade_name'],
        text=results['saving_energy.electricity_kwh'],
        orientation='h'),row=1, col=4)
    
    fig.add_trace(go.Bar(
        x=results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'],
        y=results['upgrade_name'],
        text=results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'],
        orientation='h'),row=1, col=5)

    fig.update_traces(textfont_size=10, textangle=0, textposition="outside", cliponaxis=False)
    fig.update_traces(marker_color='rgb(0, 121, 61)', marker_line_color='rgb(0, 121, 61)')
    fig.update_traces(marker_color='rgb(255, 255, 255)', marker_line_color='rgb(255, 255, 255)',row=1,col=1)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=2)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=3)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=4)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=5)
    fig.update_layout(xaxis_title="", xaxis_visible=False, yaxis_visible=True,
                      xaxis2_title="Bill<br>Savings<br>usd", xaxis2_visible=True, yaxis2_visible=False,
                      xaxis3_title="Emission<br>Savings<br>kgCO2e", xaxis3_visible=True, yaxis3_visible=False,
                      xaxis4_title="Electricity<br>Savings<br>kwh", xaxis4_visible=True, yaxis4_visible=False,
                      xaxis5_title="Gas, Propane,<br>Fuel Oil Savings<br>MMBtu", xaxis5_visible=True, yaxis5_visible=False,
                     title='Total Annual Savings for the Whole Community')
    fig.update_layout(
    autosize=False,
    width=900,
    height=500,
    font=dict(size=10),
    plot_bgcolor='rgba(0, 0, 0, 0)',)
    fig.update_xaxes(tickangle=45)
    fig.update_layout(showlegend=False)
    
    fig.show()
    fig.write_image(fig_dir / 'total_annual_savings_1.svg')    

# flavor 2 - site, electricity, and non-electricity
def figure_total_savings_2(commnity_results):
    results = commnity_results[[
        'upgrade_name',
        'saving_energy.total_mmbtu',
        'saving_energy.electricity_kwh',
        'saving_energy.natural_gas_therm',
        'saving_energy.fuel_oil_mmbtu',
        'saving_energy.propane_mmbtu',
        'saving_cbill.total_usd',
        'saving_emission.total_kgCO2e',
        'sample_weight']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'] = results['saving_energy.natural_gas_therm']*0.1 + results['saving_energy.fuel_oil_mmbtu']+ results['saving_energy.propane_mmbtu']
    results['saving_energy.electricity_kwh'] = results['saving_energy.electricity_kwh']*results['sample_weight']
    results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'] = results['saving_energy.natural_gas_fuel_oil_propane_mmbtu']*results['sample_weight']
    results['saving_cbill.total_usd'] = results['saving_cbill.total_usd']*results['sample_weight']
    results['saving_emission.total_kgCO2e'] = results['saving_emission.total_kgCO2e']*results['sample_weight']
    results['saving_energy.total_mmbtu'] = results['saving_energy.total_mmbtu']*results['sample_weight']
    
    agg_functions = {'upgrade_name': 'first',
                     'saving_energy.electricity_kwh': 'sum',
                     'saving_energy.natural_gas_fuel_oil_propane_mmbtu': 'sum', 
                      'saving_energy.total_mmbtu': 'sum', 
                     'saving_cbill.total_usd': 'sum',
                     'saving_emission.total_kgCO2e': 'sum'}
    results = results.groupby(results['upgrade_name']).aggregate(agg_functions)
    results = results.sort_values('saving_cbill.total_usd')
    
    fig = make_subplots(rows=1, cols=6, horizontal_spacing=0.12, column_widths=[0.001, 1, 1, 1, 1, 1])
    
    fig.add_trace(go.Bar(
        x=[0] * 176,
        y=results['upgrade_name'],
        text=None,
        orientation='h'), row=1, col=1)
    
    fig.add_trace(go.Bar(
        x=results['saving_cbill.total_usd'],
        y=results['upgrade_name'],
        text=results['saving_cbill.total_usd'],
        orientation='h'), row=1, col=2)
    
    fig.add_trace(go.Bar(
        x=results['saving_emission.total_kgCO2e'],
        y=results['upgrade_name'],
        text=results['saving_emission.total_kgCO2e'],
        orientation='h'),row=1, col=3)

    fig.add_trace(go.Bar(
        x=results['saving_energy.total_mmbtu'],
        y=results['upgrade_name'],
        text=results['saving_energy.total_mmbtu'],
        orientation='h'),row=1, col=4)
    
    fig.add_trace(go.Bar(
        x=results['saving_energy.electricity_kwh'],
        y=results['upgrade_name'],
        text=results['saving_energy.electricity_kwh'],
        orientation='h'),row=1, col=5)
    
    fig.add_trace(go.Bar(
        x=results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'],
        y=results['upgrade_name'],
        text=results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'],
        orientation='h'),row=1, col=6)

    fig.update_traces(textfont_size=10, textangle=0, textposition="outside", cliponaxis=False)
    fig.update_traces(marker_color='rgb(0, 121, 61)', marker_line_color='rgb(0, 121, 61)')
    fig.update_traces(marker_color='rgb(255, 255, 255)', marker_line_color='rgb(255, 255, 255)',row=1,col=1)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=2)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=3)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=4)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=5)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=6)
    fig.update_layout(xaxis_title="", xaxis_visible=False, yaxis_visible=True,
                      xaxis2_title="Bill<br>Savings<br>usd", xaxis2_visible=True, yaxis2_visible=False,
                      xaxis3_title="Site Emission<br>Saving<br>kgCO2e", xaxis3_visible=True, yaxis3_visible=False,
                      xaxis4_title="Site Energy<br>Saving<br>MMBtu", xaxis4_visible=True, yaxis4_visible=False,
                      xaxis5_title="Electricity<br>Saving<br>kwh", xaxis5_visible=True, yaxis5_visible=False,
                      xaxis6_title="Gas, Propane,<br>Fuel Oil Saving<br>MMBtu", xaxis6_visible=True, yaxis6_visible=False,
                     title='Community Total Annual Savings')
    fig.update_layout(
    autosize=False,
    width=1100,
    height=500,
    font=dict(size=10),
    plot_bgcolor='rgba(0, 0, 0, 0)',)
    fig.update_xaxes(tickangle=45)
    fig.update_layout(showlegend=False)
    
    fig.show()
    fig.write_image(fig_dir / 'total_annual_savings.svg')
    
    return results[['saving_cbill.total_usd','saving_emission.total_kgCO2e','saving_energy.total_mmbtu','saving_energy.electricity_kwh','saving_energy.natural_gas_fuel_oil_propane_mmbtu']]

# flavor 3 - site energy only
def figure_total_savings_3(commnity_results):
    results = commnity_results[[
        'upgrade_name',
        'saving_energy.total_mmbtu',
        'saving_cbill.total_usd',
        'saving_emission.total_kgCO2e',
        'sample_weight']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results['saving_energy.total_mmbtu'] = results['saving_energy.total_mmbtu']*results['sample_weight']
    results['saving_cbill.total_usd'] = results['saving_cbill.total_usd']*results['sample_weight']
    results['saving_emission.total_kgCO2e'] = results['saving_emission.total_kgCO2e']*results['sample_weight']
    
    agg_functions = {'upgrade_name': 'first',
                     'saving_energy.total_mmbtu': 'sum', 
                     'saving_cbill.total_usd': 'sum',
                     'saving_emission.total_kgCO2e': 'sum'}
    results = results.groupby(results['upgrade_name']).aggregate(agg_functions)
    results = results.sort_values('saving_cbill.total_usd')
    
    fig = make_subplots(rows=1, cols=4, horizontal_spacing=0.1, column_widths=[0.02, 1, 1, 1])
    
    fig.add_trace(go.Bar(
        x=[0] * 176,
        y=results['upgrade_name'],
        text=None,
        orientation='h'), row=1, col=1)
    
    fig.add_trace(go.Bar(
        x=results['saving_cbill.total_usd'],
        y=results['upgrade_name'],
        text=results['saving_cbill.total_usd'],
        orientation='h'), row=1, col=2)
    
    fig.add_trace(go.Bar(
        x=results['saving_emission.total_kgCO2e'],
        y=results['upgrade_name'],
        text=results['saving_emission.total_kgCO2e'],
        orientation='h'),row=1, col=3)
    
    fig.add_trace(go.Bar(
        x=results['saving_energy.total_mmbtu'],
        y=results['upgrade_name'],
        text=results['saving_energy.total_mmbtu'],
        orientation='h'),row=1, col=4)

    fig.update_traces(textfont_size=10, textangle=0, textposition="outside", cliponaxis=False)
    fig.update_traces(marker_color='rgb(0, 121, 61)', marker_line_color='rgb(0, 121, 61)')
    fig.update_traces(marker_color='rgb(255, 255, 255)', marker_line_color='rgb(255, 255, 255)',row=1,col=1)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=2)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=3)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=4)
    fig.update_layout(xaxis_title="", xaxis_visible=False, yaxis_visible=True,
                      xaxis2_title="Bill Savings<br>usd", xaxis2_visible=True, yaxis2_visible=False,
                      xaxis3_title="Emission Savings<br>kgCO2e", xaxis3_visible=True, yaxis3_visible=False,
                      xaxis4_title="Energy Savings<br>MMBtu", xaxis4_visible=True, yaxis4_visible=False,
                     title='Total Annual Savings for the Whole Community')
    fig.update_layout(
    autosize=False,
    width=850,
    height=500,
    font=dict(size=10),
    plot_bgcolor='rgba(0, 0, 0, 0)',)
    fig.update_xaxes(tickangle=45)
    fig.update_layout(showlegend=False)
    
    fig.show()
    fig.write_image(fig_dir / 'total_annual_savings_3.svg')

## Whole Community Annual Savings Per Dewlling Unit

In [5]:
def figure_mean_savings(commnity_results):
    results = commnity_results[['upgrade_name',
                       'saving_energy.total_mmbtu',
                       'saving_cbill.total_usd',
                       'saving_emission.total_kgCO2e',
                                'sample_weight']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results['saving_energy.total_mmbtu'] = results['saving_energy.total_mmbtu']*results['sample_weight']
    results['saving_cbill.total_usd'] = results['saving_cbill.total_usd']*results['sample_weight']
    results['saving_emission.total_kgCO2e'] = results['saving_emission.total_kgCO2e']*results['sample_weight']
    
    agg_functions = {'upgrade_name': 'first',
                     'saving_energy.total_mmbtu': 'sum', 
                     'saving_cbill.total_usd': 'sum',
                     'saving_emission.total_kgCO2e': 'sum',
                    'sample_weight': 'sum'}
    results = results.groupby(results['upgrade_name']).aggregate(agg_functions)
    results['saving_energy.total_mmbtu'] = results['saving_energy.total_mmbtu']/results['sample_weight']
    results['saving_cbill.total_usd'] = results['saving_cbill.total_usd']/results['sample_weight']
    results['saving_emission.total_kgCO2e'] = results['saving_emission.total_kgCO2e']/results['sample_weight']
    results = results.sort_values('saving_cbill.total_usd')
    
    print('sample_weight')
    display(results['sample_weight'])
  
    fig = make_subplots(rows=1, cols=4, horizontal_spacing=0.1, column_widths=[0.02, 1, 1, 1])
    
    fig.add_trace(go.Bar(
        x=[0] * 176,
        y=results['upgrade_name'],
        text=None,
        orientation='h'), row=1, col=1)
    
    fig.add_trace(go.Bar(
        x=results['saving_cbill.total_usd'],
        y=results['upgrade_name'],
        text=results['saving_cbill.total_usd'],
        orientation='h'), row=1, col=2)
    
    fig.add_trace(go.Bar(
        x=results['saving_emission.total_kgCO2e'],
        y=results['upgrade_name'],
        text=results['saving_emission.total_kgCO2e'],
        orientation='h'),row=1, col=3)
    
    fig.add_trace(go.Bar(
        x=results['saving_energy.total_mmbtu'],
        y=results['upgrade_name'],
        text=results['saving_energy.total_mmbtu'],
        orientation='h'),row=1, col=4)

    fig.update_traces(textfont_size=10, textangle=0, textposition="outside", cliponaxis=False)
    fig.update_traces(marker_color='rgb(0, 121, 61)', marker_line_color='rgb(0, 121, 61)')
    fig.update_traces(marker_color='rgb(255, 255, 255)', marker_line_color='rgb(255, 255, 255)',row=1,col=1)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=2)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=3)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=4)
    fig.update_layout(xaxis_title="", xaxis_visible=False, yaxis_visible=True,
                      xaxis2_title="Bill Savings<br>usd", xaxis2_visible=True, yaxis2_visible=False,
                      xaxis3_title="Emission Savings<br>kgCO2e", xaxis3_visible=True, yaxis3_visible=False,
                      xaxis4_title="Energy Savings<br>MMBtu", xaxis4_visible=True, yaxis4_visible=False,
                     title='Annual Savings Per Dwelling Unit')
    fig.update_layout(
    autosize=False,
    width=850,
    height=500,
    font=dict(size=10),
    plot_bgcolor='rgba(0, 0, 0, 0)',)
    fig.update_xaxes(tickangle=45)
    fig.update_layout(showlegend=False)
    
    fig.show()
    fig.write_image(fig_dir / 'annual_saving_per_dwelling_unit.svg')
    
    return results[['saving_cbill.total_usd','saving_emission.total_kgCO2e','saving_energy.total_mmbtu']]

## Segment Upgrade Cost Per Dwelling Unit

In [6]:
def segment_mean_upgrade_cost(commnity_results):
    results = commnity_results[['upgrade_name',
                       'upgrade_cost_usd',
                                'sample_weight',
                                'segment']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results['upgrade_cost_usd'] = results['upgrade_cost_usd']*results['sample_weight']
    
    segment1 = results.loc[results['segment'] == 'segment1']
    segment2 = results.loc[results['segment'] == 'segment2']
    
    agg_functions = {'upgrade_name': 'first',
                     'segment': 'first',
                     'upgrade_cost_usd': 'sum',
                    'sample_weight': 'sum'}
    
    segment1 = segment1.groupby(segment1['upgrade_name']).aggregate(agg_functions)
    segment2 = segment2.groupby(segment2['upgrade_name']).aggregate(agg_functions)
    
    segment1['upgrade_cost_usd'] = segment1['upgrade_cost_usd']/segment1['sample_weight']
    segment2['upgrade_cost_usd'] = segment2['upgrade_cost_usd']/segment2['sample_weight']
    
    segment1 = segment1.sort_values('upgrade_cost_usd')
    segment2 = segment2.sort_values('upgrade_cost_usd')
    
    segment1['upgrade_cost_usd'] = segment1['upgrade_cost_usd'].round(1)
    segment2['upgrade_cost_usd'] = segment2['upgrade_cost_usd'].round(1)
    
    print('segment1: upgrade cost (usd) per dwelling unit')
    display(segment1['upgrade_cost_usd'])
    print('segment2: upgrade cost (usd) per dwelling unit')
    display(segment2['upgrade_cost_usd'])
    
    return (segment1['upgrade_cost_usd'], segment2['upgrade_cost_usd'])

## Segment Savings Per Dollar Invested

In [7]:
def segment_savings_per_dollar(indicator, commnity_results):
    
    results = commnity_results[['upgrade_name', indicator, 'upgrade_cost_usd', 'sample_weight', 'segment']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results[indicator] = results[indicator]*results['sample_weight']
    results['upgrade_cost_usd'] = results['upgrade_cost_usd']*results['sample_weight']
    
    segment1 = results.loc[results['segment'] == 'segment1']
    segment2 = results.loc[results['segment'] == 'segment2']
    
    agg_functions = {'upgrade_name': 'first',
                     indicator: 'sum',
                     'upgrade_cost_usd': 'sum',
                     'sample_weight': 'sum'}
    
    segment1_agg = segment1.groupby(segment1['upgrade_name']).aggregate(agg_functions)
    segment2_agg = segment2.groupby(segment2['upgrade_name']).aggregate(agg_functions)
    
    segment1_agg[indicator] = segment1_agg[indicator]/segment1_agg['upgrade_cost_usd']
    segment2_agg[indicator] = segment2_agg[indicator]/segment2_agg['upgrade_cost_usd']
    
    segment1_agg = segment1_agg.sort_values([indicator], ascending=False)
    segment2_agg = segment2_agg.sort_values([indicator], ascending=False)
    
    segment1_agg[indicator] = segment1_agg[indicator].round(5)
    segment2_agg[indicator] = segment2_agg[indicator].round(5)
    
    print('segment1')
    display(segment1_agg[indicator])
    print('segment2')
    display(segment2_agg[indicator])
    
    return (segment1_agg[indicator],segment2_agg[indicator])

## Segment Annual Total Savings

In [8]:
def segment_annual_total_savings(indicator, commnity_results):
    
    results = commnity_results[['upgrade_name', indicator, 'sample_weight', 'segment']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results[indicator] = results[indicator]*results['sample_weight']
    
    segment1 = results.loc[results['segment'] == 'segment1']
    segment2 = results.loc[results['segment'] == 'segment2']
    
    agg_functions = {'upgrade_name': 'first',
                     indicator: 'sum',
                    'sample_weight': 'sum'}
    
    segment1_agg = segment1.groupby(segment1['upgrade_name']).aggregate(agg_functions)
    segment2_agg = segment2.groupby(segment2['upgrade_name']).aggregate(agg_functions)
    
    segment1_agg = segment1_agg.sort_values([indicator], ascending=False)
    segment2_agg = segment2_agg.sort_values([indicator], ascending=False)
    
    segment1_agg[indicator] = segment1_agg[indicator].round(1)
    segment2_agg[indicator] = segment2_agg[indicator].round(1)

    print('segment1')
    display(segment1_agg[indicator])
    print('segment2')
    display(segment2_agg[indicator])
    
    return (segment1_agg[indicator],segment2_agg[indicator])

## Segment Annual Savings Per Dwelling Unit

In [9]:
def segment_mean_savings(indicator, commnity_results):
    
    results = commnity_results[['upgrade_name', indicator, 'sample_weight', 'segment']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results[indicator] = results[indicator]*results['sample_weight']
    
    segment1 = results.loc[results['segment'] == 'segment1']
    segment2 = results.loc[results['segment'] == 'segment2']
    
    agg_functions = {'upgrade_name': 'first',
                     indicator: 'sum',
                    'sample_weight': 'sum'}
    
    segment1_agg = segment1.groupby(segment1['upgrade_name']).aggregate(agg_functions)
    segment2_agg = segment2.groupby(segment2['upgrade_name']).aggregate(agg_functions)
    
    segment1_agg[indicator] = segment1_agg[indicator]/segment1_agg['sample_weight']
    segment2_agg[indicator] = segment2_agg[indicator]/segment2_agg['sample_weight']
    
    segment1_agg = segment1_agg.sort_values([indicator], ascending=False)
    segment2_agg = segment2_agg.sort_values([indicator], ascending=False)
    
    segment1_agg[indicator] = segment1_agg[indicator].round(1)
    segment2_agg[indicator] = segment2_agg[indicator].round(1)
    
    print('segment1')
    display(segment1_agg[indicator])
    print('segment2')
    display(segment2_agg[indicator])
    
    return (segment1_agg[indicator],segment2_agg[indicator])

## Segment Percent Stock with Net Positive Savings

In [10]:
def figure_positive_negative_savings(segment, income, commnity_results):
    results = commnity_results[['upgrade_name',
                                  'saving_energy.total_mmbtu',
                                  'saving_cbill.total_usd',
                                  'saving_emission.total_kgCO2e',
                                'sample_weight',
                               'segment']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results = results.loc[results['segment'] == segment]
    results = results.groupby(results['upgrade_name'])
  
    def pos(col): 
      return col[col > 0].count()
  
    def neg(col): 
      return col[col < 0].count() 

    percentage = pd.DataFrame()

    bill = results['saving_cbill.total_usd'].agg([('negative_values', neg),('positive_values', pos)])
    emission = results['saving_emission.total_kgCO2e'].agg([('negative_values', neg),('positive_values', pos)])
    energy = results['saving_energy.total_mmbtu'].agg([('negative_values', neg),('positive_values', pos)])
    
    percentage ['cbill_n'] = bill['negative_values']/(bill['negative_values']+bill['positive_values'])
    percentage ['cbill_p'] = bill['positive_values']/(bill['negative_values']+bill['positive_values'])
    percentage ['emission_n'] = emission['negative_values']/(emission['negative_values']+emission['positive_values'])
    percentage ['emission_p'] = emission['positive_values']/(emission['negative_values']+emission['positive_values'])
    percentage ['energy_n'] = energy['negative_values']/(energy['negative_values']+energy['positive_values'])
    percentage ['energy_p'] = energy['positive_values']/(energy['negative_values']+energy['positive_values'])
    
    
    percentage = percentage.sort_values(('cbill_p'))
    upgrade_name = percentage.index
    
    fig = make_subplots(rows=1, cols=3)
    marker1= dict(color='red')
    marker2= dict(color='green')
    
    fig.add_bar(y=upgrade_name, x=percentage ['cbill_n'], text=percentage ['cbill_n'],
                orientation='h',name = "Negative",row=1, col=1, marker=marker1)
    fig.add_bar(y=upgrade_name, x=percentage ['cbill_p'],text=percentage ['cbill_p'],
                orientation='h', name = "Positive", row=1, col=1, marker=marker2)
    
    fig.add_bar(y=upgrade_name, x=percentage ['emission_n'], text=percentage ['emission_n'],
                orientation='h',showlegend=False, row=1, col=2, marker=marker1)
    fig.add_bar(y=upgrade_name, x=percentage ['emission_p'],text=percentage ['emission_p'],
                orientation='h', showlegend=False, row=1, col=2, marker=marker2)
    
    fig.add_bar(y=upgrade_name, x=percentage ['energy_n'],text=percentage ['energy_n'],
                orientation='h',showlegend=False, row=1, col=3, marker=marker1)
    fig.add_bar(y=upgrade_name, x=percentage ['energy_p'],text=percentage ['energy_p'],
                orientation='h', showlegend=False, row=1, col=3, marker=marker2)
   
  
    fig.update_layout(barmode="relative")
    fig.update_traces(texttemplate='%{text:.0%}', textposition='inside')
    
    fig.update_layout(xaxis_title="Bill", xaxis_visible=True, yaxis_visible=True,
                      xaxis2_title="Emission", xaxis2_visible=True, yaxis2_visible=False,
                      xaxis3_title="Total Energy", xaxis3_visible=True, yaxis3_visible=False,
                      font=dict(size=10))
    
    fig.update_layout(
        title='Percentage of Household in Positive and Negative Savings')
    
    fig.update_layout(
    autosize=False,
    width=1000,
    height=500,
    font=dict(size=10))
    fig.layout.xaxis1.tickformat = ',.0%'
    fig.layout.xaxis2.tickformat = ',.0%'
    fig.layout.xaxis3.tickformat = ',.0%'
    
    fig.show()
    fig.write_image(fig_dir / f'percentage_positive_saving_{segment}_{income}.svg')
    return (percentage ['cbill_p'],percentage ['emission_p'],percentage ['energy_p'])

## Segment Average Energy Burden (%) Before and after Upgrade.

In [11]:
def segment_mean_energy_burden(indicator, commnity_results):
    
    results = commnity_results[['upgrade_name', indicator, 'baseline_energy_burden_2023_cbills.%', 'sample_weight', 'segment']]
    results.replace([np.inf, -np.inf], np.nan, inplace=True)
    results.dropna(subset=['baseline_energy_burden_2023_cbills.%'], how="all", inplace=True)
    results[indicator] = results[indicator]*results['sample_weight']
    results['baseline_energy_burden_2023_cbills.%'] = results['baseline_energy_burden_2023_cbills.%']*results['sample_weight']
    
    segment1 = results.loc[results['segment'] == 'segment1']
    segment2 = results.loc[results['segment'] == 'segment2']
    
    agg_functions = {'upgrade_name': 'first',
                     indicator: 'sum',
                     'baseline_energy_burden_2023_cbills.%': 'sum',
                    'sample_weight': 'sum'}
    
    segment1_agg = segment1.groupby(segment1['upgrade_name']).aggregate(agg_functions)
    segment2_agg = segment2.groupby(segment2['upgrade_name']).aggregate(agg_functions)
    
    segment1_agg['baseline_energy_burden_2023_cbills.%'] = segment1_agg['baseline_energy_burden_2023_cbills.%']/segment1_agg['sample_weight']
    segment2_agg['baseline_energy_burden_2023_cbills.%'] = segment2_agg['baseline_energy_burden_2023_cbills.%']/segment2_agg['sample_weight']
    
    segment1_agg[indicator] = segment1_agg[indicator]/segment1_agg['sample_weight']
    segment2_agg[indicator] = segment2_agg[indicator]/segment2_agg['sample_weight']
    
    segment1_agg['energy_burden_diff'] = segment1_agg['baseline_energy_burden_2023_cbills.%'] - segment1_agg[indicator]
    segment2_agg['energy_burden_diff'] = segment2_agg['baseline_energy_burden_2023_cbills.%'] - segment2_agg[indicator]
    
    segment1_agg = segment1_agg.sort_values(['energy_burden_diff'], ascending=False)
    segment2_agg = segment2_agg.sort_values(['energy_burden_diff'], ascending=False)
     
    segment1_agg = segment1_agg[segment1_agg.upgrade_name != 'baseline']
    segment2_agg = segment2_agg[segment2_agg.upgrade_name != 'baseline']
     
    segment1_agg = segment1_agg.rename(columns={'baseline_energy_burden_2023_cbills.%': 'before', indicator: 'after'})
    segment2_agg = segment2_agg.rename(columns={'baseline_energy_burden_2023_cbills.%': 'before', indicator: 'after'})
     
    segment1_agg['before'] = segment1_agg['before'].round(1)
    segment2_agg['before'] = segment2_agg['before'].round(1)
    segment1_agg['after'] = segment1_agg['after'].round(1)
    segment2_agg['after'] = segment2_agg['after'].round(1)
    
    print ('energy burden (%) before and after upgrade')
    print('segment1')
    display(segment1_agg[['before', 'after']])
    print('segment2')
    display(segment2_agg[['before', 'after']])
    
    return(segment1_agg[['before', 'after']],segment2_agg[['before', 'after']])

In [12]:
community = 'north_birmingham'
fig_dir = Path(".").resolve() / "results" / community
community_results = pd.read_parquet(f"data_/community_building_samples_with_upgrade_cost_and_bill/{community}/processed_upgrade_results_tiered.parquet")
saving_results = pd.DataFrame()

# 1. Results for Whole Community

## 1.1 Upgrade Cost Per Dwelling Unit

In [13]:
results = community_mean_upgrade_cost(community_results)
saving_results['whole_community_upgrade_cost_mean_usd'] = results

upgrade_cost_usd per dwelling unit


upgrade_name
Electric Cooking                                                     977.8
Electric Clothes Dryer                                              1098.0
Induction Cooking                                                   1681.3
Heat Pump Clothes Dryer                                             2209.5
Heat Pump Water Heater                                              2784.0
Basic Enclosure                                                     4919.5
Enhanced Enclosure                                                  6923.8
Mininum Efficiency Heat Pump with Existing Heat Backup             13767.5
Mininum Efficiency Heat Pump with Electric Heat Backup             13870.7
Mininum Efficiency Whole Home Electrification                      14705.3
High Efficiency Heat Pump with Electric Heat Backup                24709.8
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        29055.7
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     30475.4
High Efficie

## 1.2 Total Annual Savings

In [14]:
figure_total_savings_1(community_results)

In [15]:
results = figure_total_savings_2(community_results)
saving_results[['whole_community_saving_bill_total_usd','whole_community_saving_emission_total_kgCO2e','whole_community_saving_energy_total_mmbtu','whole_community_saving_electricity_total_kwh','whole_community_saving_natural_gas_fuel_oil_propane_total_mmbtu']] = results

In [16]:
figure_total_savings_3(community_results) 

## 1.3 Annual Savings Per Dwelling Unit

In [17]:
results = figure_mean_savings(community_results) 
saving_results[['whole_community_saving_bill_mean_usd','whole_community_saving_emission_mean_kgCO2e','whole_community_saving_energy_mean_mmbtu']] = results

sample_weight


upgrade_name
Electric Clothes Dryer                                               551.443449
Induction Cooking                                                  12295.020944
Electric Cooking                                                    3699.568356
Heat Pump Clothes Dryer                                            10690.615578
Heat Pump Water Heater                                             12368.694344
Mininum Efficiency Heat Pump with Existing Heat Backup             12421.781716
Mininum Efficiency Heat Pump with Electric Heat Backup             12433.212256
Basic Enclosure                                                    12314.594893
Enhanced Enclosure                                                 12340.998249
Mininum Efficiency Whole Home Electrification                       7450.699007
High Efficiency Heat Pump with Electric Heat Backup                12433.212256
High Efficiency Whole Home Electrification                         12445.000000
Basic Enclosure + HPWH + Hi

In [18]:
metric = "saving_cbill.total_usd"
print("Community summary:")
results = get_summary(community_results, metric, lower_percentile=0.10, middle_percentile1=0.25, middle_percentile2=0.75, upper_percentile=0.90)
saving_results['whole_community_saving_bill_total_usd_wt_p0.1'] = results.iloc[:, 3]
saving_results['whole_community_saving_bill_total_usd_wt_p0.25'] = results.iloc[:, 4]
saving_results['whole_community_saving_bill_total_usd_wt_p0.75'] = results.iloc[:, 5]
saving_results['whole_community_saving_bill_total_usd_wt_p0.9'] = results.iloc[:, 6]
results

Community summary:


,n_samples,n_represented,saving_cbill.total_usd wt_mean,saving_cbill.total_usd wt_p0.1,saving_cbill.total_usd wt_p0.25,saving_cbill.total_usd wt_p0.75,saving_cbill.total_usd wt_p0.9
upgrade_name,,,,,,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,1158,12445.000000,1175.116071,394.103078,629.625502,1611.996975,2084.804720
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,1158,12445.000000,1159.416790,386.921065,606.917602,1584.457874,2071.420383
Basic Enclosure + High Efficiency Whole Home Electrification,1158,12445.000000,1156.296771,381.003816,628.266715,1574.476899,2054.959524
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,1158,12445.000000,1140.599587,380.646818,602.598162,1558.889681,2015.857187
High Efficiency Whole Home Electrification,1158,12445.000000,1024.284506,347.634864,534.538925,1368.997110,1791.731692
High Efficiency Heat Pump with Electric Heat Backup,1155,12433.212256,777.098250,183.275977,351.621997,1068.819393,1490.309589
Mininum Efficiency Whole Home Electrification,613,7450.699007,485.558448,0.000000,189.758950,743.975982,997.617755
Enhanced Enclosure,1148,12340.998249,460.756363,49.386936,142.944359,639.227845,1026.134614
Basic Enclosure,1143,12314.594893,398.841320,42.410174,117.696592,564.438690,902.914178


# 2. Results for Segments (All Income)

Segment Definition

Segment 1. Occupied, Single family detached, gas heating, all income  

Segment 2. Occupied, Single family detached, electric heating, all income   


In [19]:
income = 'all_incomes'
vacancy_status = ['Occupied']

seg1_building_type = ['Single-Family Detached']
seg1_heating_fuel = ['Natural Gas']

seg2_building_type = ['Single-Family Detached']
seg2_heating_fuel = ['Electricity']


community_results.loc[community_results['build_existing_model.vacancy_status'].isin(vacancy_status) & 
                     community_results['build_existing_model.geometry_building_type_recs'].isin(seg1_building_type) &
                      community_results['build_existing_model.heating_fuel'].isin(seg1_heating_fuel),
                     'segment'] = 'segment1'
community_results.loc[community_results['build_existing_model.vacancy_status'].isin(vacancy_status) & 
                     community_results['build_existing_model.geometry_building_type_recs'].isin(seg2_building_type) & 
                      community_results['build_existing_model.heating_fuel'].isin(seg2_heating_fuel),
                     'segment'] = 'segment2'

In [20]:
metric = 'saving_cbill.total_usd'  # TODO 'pct_saving_cbill.total_%' #'pct_saving_energy.total_%' #'saving_energy.total_mmbtu' #'saving_cbill.total_usd'

for segment in sorted(community_results["segment"].dropna().unique()):
    print(f"Summary for segment: {segment}: ")
    df_seg = community_results.loc[community_results["segment"]==segment]
    results = get_summary(df_seg, metric, lower_percentile=0.10, middle_percentile1=0.25, middle_percentile2=0.75, upper_percentile=0.90)
    saving_results[f'{segment}_{income}_saving_bill_total_usd_wt_p0.1'] = results.iloc[:, 3]
    saving_results[f'{segment}_{income}_saving_bill_total_usd_wt_p0.25'] = results.iloc[:, 4]
    saving_results[f'{segment}_{income}_saving_bill_total_usd_wt_p0.75'] = results.iloc[:, 5]
    saving_results[f'{segment}_{income}_saving_bill_total_usd_wt_p0.9'] = results.iloc[:, 6]
    display(results)

Summary for segment: nan: 


,n_samples,n_represented,saving_cbill.total_usd wt_mean,saving_cbill.total_usd wt_p0.1,saving_cbill.total_usd wt_p0.25,saving_cbill.total_usd wt_p0.75,saving_cbill.total_usd wt_p0.9
upgrade_name,,,,,,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,410,4945.494403,658.267124,253.503206,400.627697,820.408957,1149.605435
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,410,4945.494403,649.572223,240.461035,396.894229,826.905862,1134.528344
Basic Enclosure + High Efficiency Whole Home Electrification,410,4945.494403,643.998112,254.692000,383.634165,810.608499,1129.454082
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,410,4945.494403,635.303720,239.640184,385.381718,815.421196,1106.989568
High Efficiency Whole Home Electrification,410,4945.494403,575.252118,223.510469,358.181227,713.786822,979.960511
High Efficiency Heat Pump with Electric Heat Backup,407,4933.706659,404.670626,118.193551,208.342574,526.284104,726.896936
Mininum Efficiency Whole Home Electrification,142,2389.616391,234.827077,-39.661362,80.384783,379.801208,533.133157
Enhanced Enclosure,404,4895.519812,219.297242,31.057752,63.968032,315.039148,462.308780
Basic Enclosure,400,4874.921027,183.405854,25.057879,52.660875,271.378779,408.027748


Summary for segment: segment1: 


,n_samples,n_represented,saving_cbill.total_usd wt_mean,saving_cbill.total_usd wt_p0.1,saving_cbill.total_usd wt_p0.25,saving_cbill.total_usd wt_p0.75,saving_cbill.total_usd wt_p0.9
upgrade_name,,,,,,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,392,4225.727612,1500.093380,769.455731,1051.899745,1863.496449,2287.412751
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,392,4225.727612,1487.665778,785.411385,1057.958870,1834.593511,2265.991863
Basic Enclosure + High Efficiency Whole Home Electrification,392,4225.727612,1475.975915,769.455731,1030.528326,1813.911349,2237.472724
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,392,4225.727612,1463.535748,781.268491,1029.989995,1816.889551,2218.917545
High Efficiency Whole Home Electrification,392,4225.727612,1293.487901,712.545895,940.597920,1589.598999,1930.625102
High Efficiency Heat Pump with Electric Heat Backup,392,4225.727612,954.792735,362.218170,623.930226,1211.935375,1518.750765
Mininum Efficiency Whole Home Electrification,392,4225.727612,697.680570,278.998070,471.107472,909.562042,1188.388654
Enhanced Enclosure,391,4202.509328,609.838113,137.912045,299.163127,828.242459,1101.239432
Basic Enclosure,390,4196.704757,522.978059,122.399898,246.689317,714.975082,967.786354


Summary for segment: segment2: 


,n_samples,n_represented,saving_cbill.total_usd wt_mean,saving_cbill.total_usd wt_p0.1,saving_cbill.total_usd wt_p0.25,saving_cbill.total_usd wt_p0.75,saving_cbill.total_usd wt_p0.9
upgrade_name,,,,,,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,356,3273.777985,1536.413627,716.029951,968.426667,1868.786825,2573.328802
Basic Enclosure + High Efficiency Whole Home Electrification,356,3273.777985,1517.559253,709.926140,967.247142,1852.524440,2564.662594
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,356,3273.777985,1505.910244,679.596734,945.472538,1842.563089,2540.534235
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,356,3273.777985,1487.079293,679.596734,935.635408,1831.483503,2516.826741
High Efficiency Whole Home Electrification,356,3273.777985,1355.127632,654.215073,877.752957,1611.905335,2174.781222
High Efficiency Heat Pump with Electric Heat Backup,356,3273.777985,1108.996186,430.160330,601.836225,1388.713866,1911.180045
Enhanced Enclosure,353,3242.969109,632.065567,114.341272,257.076004,837.608002,1341.124291
Mininum Efficiency Heat Pump with Electric Heat Backup,356,3273.777985,595.604443,114.100929,228.912488,799.342914,1241.220886
Mininum Efficiency Heat Pump with Existing Heat Backup,356,3273.777985,595.603189,114.100929,228.912488,799.342914,1241.220886


## 2.1 Upgrade Cost Per Dwelling Unit 

In [21]:
results = segment_mean_upgrade_cost(community_results) 
saving_results[f'segment1_{income}_upgrade_cost_mean_usd'] = results[0]
saving_results[f'segment2_{income}_upgrade_cost_mean_usd'] = results[1]

segment1: upgrade cost (usd) per dwelling unit


upgrade_name
Electric Cooking                                                     977.8
Electric Clothes Dryer                                              1098.0
Induction Cooking                                                   1681.3
Heat Pump Clothes Dryer                                             2209.5
Heat Pump Water Heater                                              2891.3
Basic Enclosure                                                     6464.5
Enhanced Enclosure                                                  8852.1
Mininum Efficiency Heat Pump with Existing Heat Backup             16194.6
Mininum Efficiency Heat Pump with Electric Heat Backup             16443.3
Mininum Efficiency Whole Home Electrification                      19425.5
High Efficiency Heat Pump with Electric Heat Backup                26772.6
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        31746.5
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     33345.8
High Efficie

segment2: upgrade cost (usd) per dwelling unit


upgrade_name
Electric Cooking                                                     977.8
Electric Clothes Dryer                                              1098.0
Induction Cooking                                                   1681.3
Mininum Efficiency Whole Home Electrification                       2050.6
Heat Pump Clothes Dryer                                             2209.5
Heat Pump Water Heater                                              2896.3
Basic Enclosure                                                     5699.3
Enhanced Enclosure                                                  7654.7
Mininum Efficiency Heat Pump with Electric Heat Backup             15591.5
Mininum Efficiency Heat Pump with Existing Heat Backup             15591.5
High Efficiency Heat Pump with Electric Heat Backup                26662.5
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        31360.1
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     32694.8
High Efficie

 ## 2.2 Annual Savings Per Dollar Invested

 ### Annual Bill Savings Per Dollar Invested

In [22]:
indicator ='saving_cbill.total_usd'
results = segment_savings_per_dollar(indicator, community_results) 
saving_results[f'segment1_{income}_saving_bill_per_dollar_invested_usd'] = results[0]
saving_results[f'segment2_{income}_saving_bill_per_dollar_invested_usd'] = results[1]

segment1


upgrade_name
Basic Enclosure                                                    0.08090
Enhanced Enclosure                                                 0.06889
Heat Pump Water Heater                                             0.05427
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.04610
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.04461
Basic Enclosure + High Efficiency Whole Home Electrification       0.04153
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.04039
High Efficiency Whole Home Electrification                         0.03866
Mininum Efficiency Whole Home Electrification                      0.03592
High Efficiency Heat Pump with Electric Heat Backup                0.03566
Mininum Efficiency Heat Pump with Electric Heat Backup             0.02481
Mininum Efficiency Heat Pump with Existing Heat Backup             0.01607
Heat Pump Clothes Dryer                                            0.01526
Electric Coo

segment2


upgrade_name
Basic Enclosure                                                    0.09862
Enhanced Enclosure                                                 0.08257
Mininum Efficiency Whole Home Electrification                      0.06328
Heat Pump Water Heater                                             0.05430
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.04742
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.04606
Basic Enclosure + High Efficiency Whole Home Electrification       0.04318
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.04211
High Efficiency Heat Pump with Electric Heat Backup                0.04159
High Efficiency Whole Home Electrification                         0.04071
Mininum Efficiency Heat Pump with Electric Heat Backup             0.03820
Mininum Efficiency Heat Pump with Existing Heat Backup             0.03820
Electric Cooking                                                   0.03545
Electric Clo

 ### Annual Energy Savings Per Dollar Invested

In [23]:
indicator = 'saving_energy.total_mmbtu'
results = segment_savings_per_dollar(indicator, community_results) 
saving_results[f'segment1_{income}_saving_energy_per_dollar_invested_mmbtu'] = results[0]
saving_results[f'segment2_{income}_saving_energy_per_dollar_invested_mmbtu'] = results[1]

segment1


upgrade_name
Basic Enclosure                                                    0.00403
Enhanced Enclosure                                                 0.00343
Heat Pump Water Heater                                             0.00284
Mininum Efficiency Whole Home Electrification                      0.00251
Mininum Efficiency Heat Pump with Electric Heat Backup             0.00223
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.00218
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.00210
Basic Enclosure + High Efficiency Whole Home Electrification       0.00200
High Efficiency Whole Home Electrification                         0.00197
High Efficiency Heat Pump with Electric Heat Backup                0.00196
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.00194
Electric Cooking                                                   0.00166
Mininum Efficiency Heat Pump with Existing Heat Backup             0.00132
Induction Co

segment2


upgrade_name
Mininum Efficiency Whole Home Electrification                      0.00351
Basic Enclosure                                                    0.00292
Enhanced Enclosure                                                 0.00245
Heat Pump Water Heater                                             0.00185
Electric Cooking                                                   0.00155
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.00141
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.00137
Basic Enclosure + High Efficiency Whole Home Electrification       0.00129
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.00126
High Efficiency Heat Pump with Electric Heat Backup                0.00123
High Efficiency Whole Home Electrification                         0.00123
Mininum Efficiency Heat Pump with Electric Heat Backup             0.00115
Mininum Efficiency Heat Pump with Existing Heat Backup             0.00115
Electric Clo

 ### Annual Emission Savings Per Dollar Invested

In [24]:
indicator ='saving_emission.total_kgCO2e'
results = segment_savings_per_dollar(indicator, community_results) 
saving_results[f'segment1_{income}_saving_emission_per_dollar_invested_kgCO2e'] = results[0]
saving_results[f'segment2_{income}_saving_emission_per_dollar_invested_kgCO2e'] = results[1]

segment1


upgrade_name
Basic Enclosure                                                    0.26693
Enhanced Enclosure                                                 0.22932
Heat Pump Water Heater                                             0.18778
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.14223
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.13756
Mininum Efficiency Whole Home Electrification                      0.13729
Basic Enclosure + High Efficiency Whole Home Electrification       0.13055
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.12687
High Efficiency Whole Home Electrification                         0.12472
High Efficiency Heat Pump with Electric Heat Backup                0.12231
Mininum Efficiency Heat Pump with Electric Heat Backup             0.11393
Electric Cooking                                                   0.09686
Mininum Efficiency Heat Pump with Existing Heat Backup             0.07509
Heat Pump Cl

segment2


upgrade_name
Basic Enclosure                                                    0.27353
Mininum Efficiency Whole Home Electrification                      0.24477
Enhanced Enclosure                                                 0.23061
Heat Pump Water Heater                                             0.13367
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.12916
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.12560
Basic Enclosure + High Efficiency Whole Home Electrification       0.11797
High Efficiency Heat Pump with Electric Heat Backup                0.11717
Mininum Efficiency Heat Pump with Electric Heat Backup             0.11637
Mininum Efficiency Heat Pump with Existing Heat Backup             0.11637
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.11519
High Efficiency Whole Home Electrification                         0.11211
Electric Cooking                                                   0.10019
Heat Pump Cl

## 2.3 Annual Total Savings

### Annual Total Bill Savings

In [25]:
indicator ='saving_cbill.total_usd'
results = segment_annual_total_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_bill_total_usd'] = results[0]
saving_results[f'segment2_{income}_saving_bill_total_usd'] = results[1]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    6338986.0
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     6286470.4
Basic Enclosure + High Efficiency Whole Home Electrification       6237072.2
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        6184503.4
High Efficiency Whole Home Electrification                         5465927.5
High Efficiency Heat Pump with Electric Heat Backup                4034694.0
Mininum Efficiency Whole Home Electrification                      2948208.0
Enhanced Enclosure                                                 2562850.4
Basic Enclosure                                                    2194784.5
Mininum Efficiency Heat Pump with Electric Heat Backup             1724246.5
Mininum Efficiency Heat Pump with Existing Heat Backup             1099655.9
Heat Pump Water Heater                                              663067.4
Heat Pump Clothes Dryer                                        

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    5029877.1
Basic Enclosure + High Efficiency Whole Home Electrification       4968152.1
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     4930015.8
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        4868367.5
High Efficiency Whole Home Electrification                         4436387.0
High Efficiency Heat Pump with Electric Heat Backup                3630607.3
Enhanced Enclosure                                                 2049769.1
Mininum Efficiency Heat Pump with Electric Heat Backup             1949876.7
Mininum Efficiency Heat Pump with Existing Heat Backup             1949872.6
Basic Enclosure                                                    1822695.7
Heat Pump Water Heater                                              505237.6
Heat Pump Clothes Dryer                                             110263.4
Mininum Efficiency Whole Home Electrification                  

### Annual Total Energy Savings

In [26]:
indicator ='saving_energy.total_mmbtu'
results = segment_annual_total_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_energy_total_mmbtu'] = results[0]
saving_results[f'segment2_{income}_saving_energy_total_mmbtu'] = results[1]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    303890.4
Basic Enclosure + High Efficiency Whole Home Electrification       300939.4
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     295912.1
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        292961.1
High Efficiency Whole Home Electrification                         278650.0
High Efficiency Heat Pump with Electric Heat Backup                221488.4
Mininum Efficiency Whole Home Electrification                      206091.2
Mininum Efficiency Heat Pump with Electric Heat Backup             155075.5
Enhanced Enclosure                                                 127596.2
Basic Enclosure                                                    109290.2
Mininum Efficiency Heat Pump with Existing Heat Backup              90556.1
Heat Pump Water Heater                                              34728.1
Heat Pump Clothes Dryer                                              4222.2

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    150703.0
Basic Enclosure + High Efficiency Whole Home Electrification       148922.3
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     146336.8
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        144556.0
High Efficiency Whole Home Electrification                         133545.4
High Efficiency Heat Pump with Electric Heat Backup                107275.3
Enhanced Enclosure                                                  60767.5
Mininum Efficiency Heat Pump with Electric Heat Backup              58951.1
Mininum Efficiency Heat Pump with Existing Heat Backup              58951.0
Basic Enclosure                                                     54035.9
Heat Pump Water Heater                                              17221.3
Mininum Efficiency Whole Home Electrification                        6014.9
Heat Pump Clothes Dryer                                              3181.3

### Annual Total Emission Savings

In [27]:
indicator ='saving_emission.total_kgCO2e'
results = segment_annual_total_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_emission_total_kgCO2e'] = results[0]
saving_results[f'segment2_{income}_saving_emission_total_kgCO2e'] = results[1]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    19912155.1
Basic Enclosure + High Efficiency Whole Home Electrification       19608300.7
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     19383775.0
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        19079920.6
High Efficiency Whole Home Electrification                         17634382.5
High Efficiency Heat Pump with Electric Heat Backup                13836953.5
Mininum Efficiency Whole Home Electrification                      11269415.2
Enhanced Enclosure                                                  8530978.5
Mininum Efficiency Heat Pump with Electric Heat Backup              7916340.1
Basic Enclosure                                                     7241656.4
Mininum Efficiency Heat Pump with Existing Heat Backup              5138743.1
Heat Pump Water Heater                                              2294283.7
Heat Pump Clothes Dryer                            

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    13756931.3
Basic Enclosure + High Efficiency Whole Home Electrification       13573474.2
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     13443613.6
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        13260156.5
High Efficiency Whole Home Electrification                         12218838.8
High Efficiency Heat Pump with Electric Heat Backup                10227312.5
Mininum Efficiency Heat Pump with Electric Heat Backup              5939976.6
Mininum Efficiency Heat Pump with Existing Heat Backup              5939968.4
Enhanced Enclosure                                                  5724741.4
Basic Enclosure                                                     5055598.3
Heat Pump Water Heater                                              1243667.8
Mininum Efficiency Whole Home Electrification                        419285.1
Heat Pump Clothes Dryer                            

## 2.4 Annual Savings Per Dwelling Unit

### Annual Bill Savings Per Dwelling Unit

In [28]:
indicator ='saving_cbill.total_usd'
results = segment_mean_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_bill_mean_usd'] = results[0]
saving_results[f'segment2_{income}_saving_bill_mean_usd'] = results[1]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    1500.1
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     1487.7
Basic Enclosure + High Efficiency Whole Home Electrification       1476.0
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        1463.5
High Efficiency Whole Home Electrification                         1293.5
High Efficiency Heat Pump with Electric Heat Backup                 954.8
Mininum Efficiency Whole Home Electrification                       697.7
Enhanced Enclosure                                                  609.8
Basic Enclosure                                                     523.0
Mininum Efficiency Heat Pump with Electric Heat Backup              408.0
Mininum Efficiency Heat Pump with Existing Heat Backup              260.2
Heat Pump Water Heater                                              156.9
Heat Pump Clothes Dryer                                              33.7
Electric Cooking         

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    1536.4
Basic Enclosure + High Efficiency Whole Home Electrification       1517.6
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     1505.9
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        1487.1
High Efficiency Whole Home Electrification                         1355.1
High Efficiency Heat Pump with Electric Heat Backup                1109.0
Enhanced Enclosure                                                  632.1
Mininum Efficiency Heat Pump with Electric Heat Backup              595.6
Mininum Efficiency Heat Pump with Existing Heat Backup              595.6
Basic Enclosure                                                     562.0
Heat Pump Water Heater                                              157.3
Mininum Efficiency Whole Home Electrification                       129.8
Heat Pump Clothes Dryer                                              35.0
Electric Clothes Dryer   

### Annual Energy Savings Per Dwelling Unit

In [29]:
indicator ='saving_energy.total_mmbtu'
results = segment_mean_savings(indicator, community_results) 
saving_results[f'segment1_{income}_saving_energy_mean_mmbtu'] = results[0]
saving_results[f'segment2_{income}_saving_energy_mean_mmbtu'] = results[1]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    71.9
Basic Enclosure + High Efficiency Whole Home Electrification       71.2
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     70.0
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        69.3
High Efficiency Whole Home Electrification                         65.9
High Efficiency Heat Pump with Electric Heat Backup                52.4
Mininum Efficiency Whole Home Electrification                      48.8
Mininum Efficiency Heat Pump with Electric Heat Backup             36.7
Enhanced Enclosure                                                 30.4
Basic Enclosure                                                    26.0
Mininum Efficiency Heat Pump with Existing Heat Backup             21.4
Heat Pump Water Heater                                              8.2
Electric Cooking                                                    1.6
Heat Pump Clothes Dryer                            

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    46.0
Basic Enclosure + High Efficiency Whole Home Electrification       45.5
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     44.7
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        44.2
High Efficiency Whole Home Electrification                         40.8
High Efficiency Heat Pump with Electric Heat Backup                32.8
Enhanced Enclosure                                                 18.7
Mininum Efficiency Heat Pump with Electric Heat Backup             18.0
Mininum Efficiency Heat Pump with Existing Heat Backup             18.0
Basic Enclosure                                                    16.7
Mininum Efficiency Whole Home Electrification                       7.2
Heat Pump Water Heater                                              5.4
Electric Cooking                                                    1.5
Heat Pump Clothes Dryer                            

### Annual Percentage Energy Savings Per Dwelling Unit

In [30]:
indicator ='pct_saving_energy.total_%'
results = segment_mean_savings(indicator, community_results)
saving_results[f'segment1_{income}_pct_saving_energy_mean_%'] = results[0]
saving_results[f'segment2_{income}_pct_saving_energy_mean_%'] = results[1]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    64.5
Basic Enclosure + High Efficiency Whole Home Electrification       63.9
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     62.6
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        62.0
High Efficiency Whole Home Electrification                         59.2
High Efficiency Heat Pump with Electric Heat Backup                45.9
Mininum Efficiency Whole Home Electrification                      43.3
Mininum Efficiency Heat Pump with Electric Heat Backup             31.5
Enhanced Enclosure                                                 26.4
Basic Enclosure                                                    22.7
Mininum Efficiency Heat Pump with Existing Heat Backup             17.6
Heat Pump Water Heater                                              8.2
Electric Cooking                                                    1.7
Heat Pump Clothes Dryer                            

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    53.0
Basic Enclosure + High Efficiency Whole Home Electrification       52.4
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     51.3
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        50.6
High Efficiency Whole Home Electrification                         47.1
High Efficiency Heat Pump with Electric Heat Backup                37.0
Enhanced Enclosure                                                 20.5
Mininum Efficiency Heat Pump with Electric Heat Backup             19.4
Mininum Efficiency Heat Pump with Existing Heat Backup             19.4
Basic Enclosure                                                    18.1
Mininum Efficiency Whole Home Electrification                       8.1
Heat Pump Water Heater                                              6.6
Electric Cooking                                                    2.1
Heat Pump Clothes Dryer                            

### Annual Emission Savings Per Dwelling Unit

In [31]:
indicator ='saving_emission.total_kgCO2e'
results = segment_mean_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_emission_mean_kgCO2e'] = results[0]
saving_results[f'segment2_{income}_saving_emission_mean_kgCO2e'] = results[1]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    4712.1
Basic Enclosure + High Efficiency Whole Home Electrification       4640.2
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     4587.1
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        4515.2
High Efficiency Whole Home Electrification                         4173.1
High Efficiency Heat Pump with Electric Heat Backup                3274.5
Mininum Efficiency Whole Home Electrification                      2666.9
Enhanced Enclosure                                                 2030.0
Mininum Efficiency Heat Pump with Electric Heat Backup             1873.4
Basic Enclosure                                                    1725.6
Mininum Efficiency Heat Pump with Existing Heat Backup             1216.1
Heat Pump Water Heater                                              542.9
Electric Cooking                                                     94.7
Heat Pump Clothes Dryer  

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    4202.2
Basic Enclosure + High Efficiency Whole Home Electrification       4146.1
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     4106.5
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        4050.4
High Efficiency Whole Home Electrification                         3732.3
High Efficiency Heat Pump with Electric Heat Backup                3124.0
Mininum Efficiency Heat Pump with Electric Heat Backup             1814.4
Mininum Efficiency Heat Pump with Existing Heat Backup             1814.4
Enhanced Enclosure                                                 1765.3
Basic Enclosure                                                    1558.9
Mininum Efficiency Whole Home Electrification                       501.9
Heat Pump Water Heater                                              387.1
Electric Cooking                                                     98.0
Heat Pump Clothes Dryer  

## 2.5. Pecent Stock with Positive and Negtive Savings

### Segment 1

In [32]:
segment = 'segment1'
results = figure_positive_negative_savings(segment, income, community_results) 
saving_results[f'segment1_{income}_pct_bill_positive_savings_%'] = results[0]*100
saving_results[f'segment1_{income}_pct_emission_positive_savings_%'] = results[1]*100
saving_results[f'segment1_{income}_pct_energy_positive_savings_%'] = results[2]*100

### Segment 2

In [33]:
segment = 'segment2'
results = figure_positive_negative_savings(segment, income, community_results)
saving_results[f'segment2_{income}_pct_bill_positive_savings_%'] = results[0]*100
saving_results[f'segment2_{income}_pct_emission_positive_savings_%'] = results[1]*100
saving_results[f'segment2_{income}_pct_energy_positive_savings_%'] = results[2]*100

## 2.6. Average Energy Burden (%) Before and after Upgrade

In [34]:
results = segment_mean_energy_burden('post-upgrade_energy_burden_2023_cbills.%', community_results) 
saving_results[[f'segment1_{income}_energy_burden_before_upgrade_%',f'segment1_{income}_energy_burden_after_upgrade_%']] = results[0]
saving_results[[f'segment2_{income}_energy_burden_before_upgrade_%',f'segment2_{income}_energy_burden_after_upgrade_%']] = results[1]

energy burden (%) before and after upgrade
segment1


/var/folders/qt/5nrnlf6148z2jxyxx_5yqgl0z8pq2p/T/ipykernel_79926/2800418672.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/qt/5nrnlf6148z2jxyxx_5yqgl0z8pq2p/T/ipykernel_79926/2800418672.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/qt/5nrnlf6148z2jxyxx_5yqgl0z8pq2p/T/ipykernel_79926/2800418672.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

,before,after
upgrade_name,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,8.4,4.1
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,8.4,4.2
Basic Enclosure + High Efficiency Whole Home Electrification,8.4,4.2
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,8.4,4.2
High Efficiency Whole Home Electrification,8.4,4.7
High Efficiency Heat Pump with Electric Heat Backup,8.4,5.7
Mininum Efficiency Whole Home Electrification,8.4,6.4
Enhanced Enclosure,8.4,6.6
Basic Enclosure,8.4,6.8


segment2


,before,after
upgrade_name,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,10.1,4.7
Basic Enclosure + High Efficiency Whole Home Electrification,10.1,4.8
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,10.1,4.8
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,10.1,4.8
High Efficiency Whole Home Electrification,10.1,5.4
High Efficiency Heat Pump with Electric Heat Backup,10.1,6.1
Enhanced Enclosure,10.1,7.8
Basic Enclosure,10.1,7.9
Mininum Efficiency Heat Pump with Electric Heat Backup,10.1,7.9


# 3. Results for Segments (0-80% AMI)

Segment Definition

Segment 1. Occupied, Single family detached, gas heating, 0-80% AMI  

Segment 2. Occupied, Single family detached, electric heating, 0-80% AMI      

In [35]:
community_results = pd.read_parquet(f"data_/community_building_samples_with_upgrade_cost_and_bill/{community}/processed_upgrade_results_tiered.parquet")

income = '0-80%AMI'
vacancy_status = ['Occupied']

seg1_building_type = ['Single-Family Detached']
seg1_heating_fuel = ['Natural Gas']
seg1_median_income = ['0-30%','30-60%','60-80%']

seg2_building_type = ['Single-Family Detached']
seg2_heating_fuel = ['Electricity']
seg2_median_income = ['0-30%','30-60%','60-80%']


community_results.loc[community_results['build_existing_model.vacancy_status'].isin(vacancy_status) & 
                     community_results['build_existing_model.geometry_building_type_recs'].isin(seg1_building_type) & 
                      community_results['build_existing_model.heating_fuel'].isin(seg1_heating_fuel) &
                     community_results['build_existing_model.area_median_income'].isin(seg1_median_income),
                     'segment'] = 'segment1'
community_results.loc[community_results['build_existing_model.vacancy_status'].isin(vacancy_status) & 
                     community_results['build_existing_model.geometry_building_type_recs'].isin(seg2_building_type) &
                      community_results['build_existing_model.heating_fuel'].isin(seg2_heating_fuel) &
                     community_results['build_existing_model.area_median_income'].isin(seg2_median_income),
                     'segment'] = 'segment2'

In [36]:
metric = 'saving_cbill.total_usd'  # TODO 'pct_saving_cbill.total_%' #'pct_saving_energy.total_%' #'saving_energy.total_mmbtu' #'saving_cbill.total_usd'

for segment in sorted(community_results["segment"].dropna().unique()):
    print(f"Summary for segment: {segment}: ")
    df_seg = community_results.loc[community_results["segment"]==segment]
    results = get_summary(df_seg, metric, lower_percentile=0.10, middle_percentile1=0.25, middle_percentile2=0.75, upper_percentile=0.90)
    saving_results[f'{segment}_{income}_saving_bill_total_usd_wt_p0.1'] = results.iloc[:, 3]
    saving_results[f'{segment}_{income}_saving_bill_total_usd_wt_p0.25'] = results.iloc[:, 4]
    saving_results[f'{segment}_{income}_saving_bill_total_usd_wt_p0.75'] = results.iloc[:, 5]
    saving_results[f'{segment}_{income}_saving_bill_total_usd_wt_p0.9'] = results.iloc[:, 6]
    display(results)

Summary for segment: nan: 


,n_samples,n_represented,saving_cbill.total_usd wt_mean,saving_cbill.total_usd wt_p0.1,saving_cbill.total_usd wt_p0.25,saving_cbill.total_usd wt_p0.75,saving_cbill.total_usd wt_p0.9
upgrade_name,,,,,,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,936,9552.001866,1103.494266,3.515703e+02,556.193866,1456.885576,2026.734608
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,936,9552.001866,1088.197359,3.341119e+02,555.879296,1429.586821,2014.996404
Basic Enclosure + High Efficiency Whole Home Electrification,936,9552.001866,1084.483330,3.411636e+02,543.950864,1417.811676,1984.465342
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,936,9552.001866,1069.187997,3.291457e+02,543.980918,1410.519839,1958.340684
High Efficiency Whole Home Electrification,936,9552.001866,966.655486,3.020733e+02,496.939548,1277.255334,1766.832619
High Efficiency Heat Pump with Electric Heat Backup,933,9540.214122,727.844042,1.678641e+02,313.074287,975.912234,1454.088192
Mininum Efficiency Whole Home Electrification,457,5407.490052,468.623480,-1.538479e-13,168.269139,733.298279,1006.119360
Enhanced Enclosure,926,9448.000114,414.740609,4.044700e+01,113.725675,573.417270,960.501334
Basic Enclosure,921,9421.596759,354.946582,3.608277e+01,88.533163,488.501564,848.974398


Summary for segment: segment1: 


,n_samples,n_represented,saving_cbill.total_usd wt_mean,saving_cbill.total_usd wt_p0.1,saving_cbill.total_usd wt_p0.25,saving_cbill.total_usd wt_p0.75,saving_cbill.total_usd wt_p0.9
upgrade_name,,,,,,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,134,1699.191387,1352.128972,685.272188,998.359713,1640.848986,2128.923770
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,134,1699.191387,1341.001967,682.271373,988.933448,1646.437334,2087.915568
Basic Enclosure + High Efficiency Whole Home Electrification,134,1699.191387,1333.627134,682.744540,981.296926,1616.869375,2081.207047
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,134,1699.191387,1322.494023,682.271373,971.968994,1591.985854,2044.621152
High Efficiency Whole Home Electrification,134,1699.191387,1159.877519,577.934498,859.771543,1400.546964,1798.578483
High Efficiency Heat Pump with Electric Heat Backup,134,1699.191387,840.506259,241.749884,558.692336,1069.601117,1466.434737
Mininum Efficiency Whole Home Electrification,134,1699.191387,617.767065,175.460583,426.854317,804.687055,1056.931170
Enhanced Enclosure,134,1699.191387,564.621797,211.588833,297.021540,733.989391,1069.816531
Basic Enclosure,134,1699.191387,491.307100,158.586311,251.601503,655.290063,971.514837


Summary for segment: segment2: 


,n_samples,n_represented,saving_cbill.total_usd wt_mean,saving_cbill.total_usd wt_p0.1,saving_cbill.total_usd wt_p0.25,saving_cbill.total_usd wt_p0.75,saving_cbill.total_usd wt_p0.9
upgrade_name,,,,,,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,88,1193.806748,1496.234054,705.102075,1028.644425,1863.047420,2230.453494
Basic Enclosure + High Efficiency Whole Home Electrification,88,1193.806748,1478.496237,705.102075,987.593106,1857.951349,2170.403709
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,88,1193.806748,1470.807365,674.842695,996.224013,1844.282155,2194.703755
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,88,1193.806748,1453.087502,674.718597,955.853555,1834.712031,2134.905615
High Efficiency Whole Home Electrification,88,1193.806748,1292.396598,620.229497,892.370525,1597.707293,1906.563619
High Efficiency Heat Pump with Electric Heat Backup,88,1193.806748,1080.458358,460.650399,596.837461,1460.831644,1681.195085
Enhanced Enclosure,88,1193.806748,677.097569,190.840288,324.195245,923.424226,1247.765042
Basic Enclosure,88,1193.806748,613.651181,162.340817,279.901342,880.214951,1073.038964
Mininum Efficiency Heat Pump with Electric Heat Backup,88,1193.806748,564.688765,88.118742,264.587290,855.238518,1127.170744


## 3.1 Upgrade Cost Per Dwelling Unit 

In [37]:
results = segment_mean_upgrade_cost(community_results) 
saving_results[f'segment1_{income}_upgrade_cost_mean_usd'] = results[0]
saving_results[f'segment2_{income}_upgrade_cost_mean_usd'] = results[1]

segment1: upgrade cost (usd) per dwelling unit


upgrade_name
Electric Cooking                                                     977.8
Electric Clothes Dryer                                              1098.0
Induction Cooking                                                   1681.3
Heat Pump Clothes Dryer                                             2209.5
Heat Pump Water Heater                                              2786.2
Basic Enclosure                                                     5946.2
Enhanced Enclosure                                                  8143.3
Mininum Efficiency Heat Pump with Existing Heat Backup             14267.0
Mininum Efficiency Heat Pump with Electric Heat Backup             14488.1
Mininum Efficiency Whole Home Electrification                      17387.3
High Efficiency Heat Pump with Electric Heat Backup                25586.2
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        30315.5
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     31749.5
High Efficie

segment2: upgrade cost (usd) per dwelling unit


upgrade_name
Electric Cooking                                                     977.8
Electric Clothes Dryer                                              1098.0
Induction Cooking                                                   1681.3
Mininum Efficiency Whole Home Electrification                       1858.5
Heat Pump Clothes Dryer                                             2209.5
Heat Pump Water Heater                                              2745.7
Basic Enclosure                                                     5885.8
Enhanced Enclosure                                                  7798.9
Mininum Efficiency Heat Pump with Electric Heat Backup             14844.7
Mininum Efficiency Heat Pump with Existing Heat Backup             14844.7
High Efficiency Heat Pump with Electric Heat Backup                25993.9
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        30603.3
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     31935.0
High Efficie

 ## 3.2 Annual Savings Per Dollar Invested

### Annual Bill Savings Per Dollar Invested

In [38]:
indicator ='saving_cbill.total_usd'
results = segment_savings_per_dollar(indicator, community_results) 
saving_results[f'segment1_{income}_saving_bill_per_dollar_invested_usd'] = results[0]
saving_results[f'segment2_{income}_saving_bill_per_dollar_invested_usd'] = results[1]

segment1


upgrade_name
Basic Enclosure                                                    0.08263
Enhanced Enclosure                                                 0.06934
Heat Pump Water Heater                                             0.05332
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.04362
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.04224
Basic Enclosure + High Efficiency Whole Home Electrification       0.03912
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.03807
High Efficiency Whole Home Electrification                         0.03608
Mininum Efficiency Whole Home Electrification                      0.03553
High Efficiency Heat Pump with Electric Heat Backup                0.03285
Mininum Efficiency Heat Pump with Electric Heat Backup             0.02379
Heat Pump Clothes Dryer                                            0.01461
Mininum Efficiency Heat Pump with Existing Heat Backup             0.01450
Electric Coo

segment2


upgrade_name
Basic Enclosure                                                    0.10426
Enhanced Enclosure                                                 0.08682
Mininum Efficiency Whole Home Electrification                      0.05313
Heat Pump Water Heater                                             0.04942
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.04748
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.04606
Basic Enclosure + High Efficiency Whole Home Electrification       0.04299
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.04188
High Efficiency Heat Pump with Electric Heat Backup                0.04157
High Efficiency Whole Home Electrification                         0.03979
Mininum Efficiency Heat Pump with Electric Heat Backup             0.03804
Mininum Efficiency Heat Pump with Existing Heat Backup             0.03804
Electric Cooking                                                   0.02086
Heat Pump Cl

 ### Annual Energy Savings Per Dollar Invested

In [39]:
indicator = 'saving_energy.total_mmbtu'
results = segment_savings_per_dollar(indicator, community_results) 
saving_results[f'segment1_{income}_saving_energy_per_dollar_invested_mmbtu'] = results[0]
saving_results[f'segment2_{income}_saving_energy_per_dollar_invested_mmbtu'] = results[1]

segment1


upgrade_name
Basic Enclosure                                                    0.00403
Enhanced Enclosure                                                 0.00338
Heat Pump Water Heater                                             0.00296
Mininum Efficiency Whole Home Electrification                      0.00243
Mininum Efficiency Heat Pump with Electric Heat Backup             0.00212
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.00202
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.00194
Basic Enclosure + High Efficiency Whole Home Electrification       0.00185
High Efficiency Whole Home Electrification                         0.00180
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.00179
High Efficiency Heat Pump with Electric Heat Backup                0.00175
Electric Cooking                                                   0.00172
Mininum Efficiency Heat Pump with Existing Heat Backup             0.00117
Induction Co

segment2


upgrade_name
Mininum Efficiency Whole Home Electrification                      0.00313
Basic Enclosure                                                    0.00309
Enhanced Enclosure                                                 0.00257
Heat Pump Water Heater                                             0.00170
Electric Cooking                                                   0.00145
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.00141
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.00137
Basic Enclosure + High Efficiency Whole Home Electrification       0.00129
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.00125
High Efficiency Heat Pump with Electric Heat Backup                0.00123
High Efficiency Whole Home Electrification                         0.00120
Mininum Efficiency Heat Pump with Electric Heat Backup             0.00115
Mininum Efficiency Heat Pump with Existing Heat Backup             0.00115
Heat Pump Cl

 ### Annual Emission Savings Per Dollar Invested

In [40]:
indicator ='saving_emission.total_kgCO2e'
results = segment_savings_per_dollar(indicator, community_results) 
saving_results[f'segment1_{income}_saving_emission_per_dollar_invested_kgCO2e'] = results[0]
saving_results[f'segment2_{income}_saving_emission_per_dollar_invested_kgCO2e'] = results[1]

segment1


upgrade_name
Basic Enclosure                                                    0.26564
Enhanced Enclosure                                                 0.22510
Heat Pump Water Heater                                             0.19703
Mininum Efficiency Whole Home Electrification                      0.13307
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.13170
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.12751
Basic Enclosure + High Efficiency Whole Home Electrification       0.12062
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.11733
High Efficiency Whole Home Electrification                         0.11421
High Efficiency Heat Pump with Electric Heat Backup                0.10955
Mininum Efficiency Heat Pump with Electric Heat Backup             0.10663
Electric Cooking                                                   0.10047
Mininum Efficiency Heat Pump with Existing Heat Backup             0.06614
Induction Co

segment2


upgrade_name
Basic Enclosure                                                    0.28833
Enhanced Enclosure                                                 0.24162
Mininum Efficiency Whole Home Electrification                      0.22435
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.12934
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.12561
Heat Pump Water Heater                                             0.12374
Basic Enclosure + High Efficiency Whole Home Electrification       0.11761
High Efficiency Heat Pump with Electric Heat Backup                0.11678
Mininum Efficiency Heat Pump with Electric Heat Backup             0.11552
Mininum Efficiency Heat Pump with Existing Heat Backup             0.11552
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.11471
High Efficiency Whole Home Electrification                         0.10988
Electric Cooking                                                   0.09272
Heat Pump Cl

## 3.3 Annual Total Savings

### Annual Total Bill Savings

In [41]:
saving_results.columns.values

array(['whole_community_upgrade_cost_mean_usd',
       'whole_community_saving_bill_total_usd',
       'whole_community_saving_emission_total_kgCO2e',
       'whole_community_saving_energy_total_mmbtu',
       'whole_community_saving_electricity_total_kwh',
       'whole_community_saving_natural_gas_fuel_oil_propane_total_mmbtu',
       'whole_community_saving_bill_mean_usd',
       'whole_community_saving_emission_mean_kgCO2e',
       'whole_community_saving_energy_mean_mmbtu',
       'whole_community_saving_bill_total_usd_wt_p0.1',
       'whole_community_saving_bill_total_usd_wt_p0.25',
       'whole_community_saving_bill_total_usd_wt_p0.75',
       'whole_community_saving_bill_total_usd_wt_p0.9',
       'nan_all_incomes_saving_bill_total_usd_wt_p0.1',
       'nan_all_incomes_saving_bill_total_usd_wt_p0.25',
       'nan_all_incomes_saving_bill_total_usd_wt_p0.75',
       'nan_all_incomes_saving_bill_total_usd_wt_p0.9',
       'segment1_all_incomes_saving_bill_total_usd_wt_p0.1',
   

In [42]:
indicator ='saving_cbill.total_usd'
results = segment_annual_total_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_bill_total_usd'] = results[0]
saving_results[f'segment2_{income}_saving_bill_total_usd'] = results[1]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    2297525.9
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     2278619.0
Basic Enclosure + High Efficiency Whole Home Electrification       2266087.7
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        2247170.5
High Efficiency Whole Home Electrification                         1970853.9
High Efficiency Heat Pump with Electric Heat Backup                1428181.0
Mininum Efficiency Whole Home Electrification                      1049704.5
Enhanced Enclosure                                                  959400.5
Basic Enclosure                                                     834824.8
Mininum Efficiency Heat Pump with Electric Heat Backup              585762.8
Mininum Efficiency Heat Pump with Existing Heat Backup              351491.8
Heat Pump Water Heater                                              252423.6
Heat Pump Clothes Dryer                                        

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    1786214.3
Basic Enclosure + High Efficiency Whole Home Electrification       1765038.8
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     1755859.8
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        1734705.7
High Efficiency Whole Home Electrification                         1542871.8
High Efficiency Heat Pump with Electric Heat Backup                1289858.5
Enhanced Enclosure                                                  808323.6
Basic Enclosure                                                     732580.9
Mininum Efficiency Heat Pump with Electric Heat Backup              674129.3
Mininum Efficiency Heat Pump with Existing Heat Backup              674126.4
Heat Pump Water Heater                                              159160.4
Heat Pump Clothes Dryer                                              33969.0
Mininum Efficiency Whole Home Electrification                  

### Annual Total Energy Savings

In [43]:
indicator ='saving_energy.total_mmbtu'
results = segment_annual_total_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_energy_total_mmbtu'] = results[0]
saving_results[f'segment2_{income}_saving_energy_total_mmbtu'] = results[1]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    107840.3
Basic Enclosure + High Efficiency Whole Home Electrification       106931.4
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     104783.9
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        103875.1
High Efficiency Whole Home Electrification                          98425.2
High Efficiency Heat Pump with Electric Heat Backup                 76297.2
Mininum Efficiency Whole Home Electrification                       71933.2
Mininum Efficiency Heat Pump with Electric Heat Backup              52134.4
Enhanced Enclosure                                                  46796.6
Basic Enclosure                                                     40688.1
Mininum Efficiency Heat Pump with Existing Heat Backup              28469.3
Heat Pump Water Heater                                              14001.0
Heat Pump Clothes Dryer                                              1601.2

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    53502.8
Basic Enclosure + High Efficiency Whole Home Electrification       52895.0
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     52062.3
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        51454.5
High Efficiency Whole Home Electrification                         46478.1
High Efficiency Heat Pump with Electric Heat Backup                38125.0
Enhanced Enclosure                                                 23952.7
Basic Enclosure                                                    21711.3
Mininum Efficiency Heat Pump with Electric Heat Backup             20404.0
Mininum Efficiency Heat Pump with Existing Heat Backup             20403.9
Heat Pump Water Heater                                              5471.7
Mininum Efficiency Whole Home Electrification                       2000.4
Heat Pump Clothes Dryer                                              996.9
Induction Co

### Annual Total Emission Savings

In [44]:
indicator ='saving_emission.total_kgCO2e'
results = segment_annual_total_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_emission_total_kgCO2e'] = results[0]
saving_results[f'segment2_{income}_saving_emission_total_kgCO2e'] = results[1]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    7081392.8
Basic Enclosure + High Efficiency Whole Home Electrification       6986142.6
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     6879139.4
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        6783889.2
High Efficiency Whole Home Electrification                         6237861.5
High Efficiency Heat Pump with Electric Heat Backup                4762953.5
Mininum Efficiency Whole Home Electrification                      3931561.7
Enhanced Enclosure                                                 3114701.1
Basic Enclosure                                                    2683993.3
Mininum Efficiency Heat Pump with Electric Heat Backup             2625139.1
Mininum Efficiency Heat Pump with Existing Heat Backup             1603328.1
Heat Pump Water Heater                                              932770.4
Heat Pump Clothes Dryer                                        

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    4892406.5
Basic Enclosure + High Efficiency Whole Home Electrification       4829079.8
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     4788849.4
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        4725522.7
High Efficiency Whole Home Electrification                         4260853.7
High Efficiency Heat Pump with Electric Heat Backup                3623736.3
Enhanced Enclosure                                                 2249525.2
Mininum Efficiency Heat Pump with Electric Heat Backup             2047279.8
Mininum Efficiency Heat Pump with Existing Heat Backup             2047274.2
Basic Enclosure                                                    2025927.6
Heat Pump Water Heater                                              398502.5
Mininum Efficiency Whole Home Electrification                       143441.3
Heat Pump Clothes Dryer                                        

## 3.4 Annual Savings Per Dwelling Unit

### Annual Bill Savings Per Dwelling Unit

In [45]:
indicator ='saving_cbill.total_usd'
results = segment_mean_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_bill_mean_usd'] = results[0]
saving_results[f'segment2_{income}_saving_bill_mean_usd'] = results[1]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    1352.1
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     1341.0
Basic Enclosure + High Efficiency Whole Home Electrification       1333.6
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        1322.5
High Efficiency Whole Home Electrification                         1159.9
High Efficiency Heat Pump with Electric Heat Backup                 840.5
Mininum Efficiency Whole Home Electrification                       617.8
Enhanced Enclosure                                                  564.6
Basic Enclosure                                                     491.3
Mininum Efficiency Heat Pump with Electric Heat Backup              344.7
Mininum Efficiency Heat Pump with Existing Heat Backup              206.9
Heat Pump Water Heater                                              148.6
Heat Pump Clothes Dryer                                              32.3
Electric Cooking         

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    1496.2
Basic Enclosure + High Efficiency Whole Home Electrification       1478.5
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     1470.8
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        1453.1
High Efficiency Whole Home Electrification                         1292.4
High Efficiency Heat Pump with Electric Heat Backup                1080.5
Enhanced Enclosure                                                  677.1
Basic Enclosure                                                     613.7
Mininum Efficiency Heat Pump with Electric Heat Backup              564.7
Mininum Efficiency Heat Pump with Existing Heat Backup              564.7
Heat Pump Water Heater                                              135.7
Mininum Efficiency Whole Home Electrification                        98.7
Heat Pump Clothes Dryer                                              29.8
Electric Cooking         

### Annual Energy Savings Per Dwelling Unit

In [46]:
indicator ='saving_energy.total_mmbtu'
results = segment_mean_savings(indicator, community_results) 
saving_results[f'segment1_{income}_saving_energy_mean_mmbtu'] = results[0]
saving_results[f'segment2_{income}_saving_energy_mean_mmbtu'] = results[1]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    63.5
Basic Enclosure + High Efficiency Whole Home Electrification       62.9
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     61.7
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        61.1
High Efficiency Whole Home Electrification                         57.9
High Efficiency Heat Pump with Electric Heat Backup                44.9
Mininum Efficiency Whole Home Electrification                      42.3
Mininum Efficiency Heat Pump with Electric Heat Backup             30.7
Enhanced Enclosure                                                 27.5
Basic Enclosure                                                    23.9
Mininum Efficiency Heat Pump with Existing Heat Backup             16.8
Heat Pump Water Heater                                              8.2
Electric Cooking                                                    1.7
Heat Pump Clothes Dryer                            

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    44.8
Basic Enclosure + High Efficiency Whole Home Electrification       44.3
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     43.6
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        43.1
High Efficiency Whole Home Electrification                         38.9
High Efficiency Heat Pump with Electric Heat Backup                31.9
Enhanced Enclosure                                                 20.1
Basic Enclosure                                                    18.2
Mininum Efficiency Heat Pump with Electric Heat Backup             17.1
Mininum Efficiency Heat Pump with Existing Heat Backup             17.1
Mininum Efficiency Whole Home Electrification                       5.8
Heat Pump Water Heater                                              4.7
Electric Cooking                                                    1.4
Heat Pump Clothes Dryer                            

### Annual Percentage Energy Savings Per Dwelling Unit

In [47]:
indicator ='pct_saving_energy.total_%'
results = segment_mean_savings(indicator, community_results)
saving_results[f'segment1_{income}_pct_saving_energy_mean_%'] = results[0]
saving_results[f'segment2_{income}_pct_saving_energy_mean_%'] = results[1]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    63.5
Basic Enclosure + High Efficiency Whole Home Electrification       63.0
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     61.5
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        61.0
High Efficiency Whole Home Electrification                         58.0
High Efficiency Heat Pump with Electric Heat Backup                44.0
Mininum Efficiency Whole Home Electrification                      41.8
Mininum Efficiency Heat Pump with Electric Heat Backup             29.5
Enhanced Enclosure                                                 26.8
Basic Enclosure                                                    23.4
Mininum Efficiency Heat Pump with Existing Heat Backup             15.2
Heat Pump Water Heater                                              8.8
Electric Cooking                                                    1.9
Heat Pump Clothes Dryer                            

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    55.1
Basic Enclosure + High Efficiency Whole Home Electrification       54.4
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     53.4
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        52.7
High Efficiency Whole Home Electrification                         47.9
High Efficiency Heat Pump with Electric Heat Backup                38.9
Enhanced Enclosure                                                 23.8
Basic Enclosure                                                    21.4
Mininum Efficiency Heat Pump with Electric Heat Backup             20.1
Mininum Efficiency Heat Pump with Existing Heat Backup             20.1
Mininum Efficiency Whole Home Electrification                       7.3
Heat Pump Water Heater                                              5.9
Electric Cooking                                                    2.1
Heat Pump Clothes Dryer                            

### Annual Emission Savings Per Dwelling Unit

In [48]:
indicator ='saving_emission.total_kgCO2e'
results = segment_mean_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_emission_mean_kgCO2e'] = results[0]
saving_results[f'segment2_{income}_saving_emission_mean_kgCO2e'] = results[1]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    4167.5
Basic Enclosure + High Efficiency Whole Home Electrification       4111.5
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     4048.5
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        3992.4
High Efficiency Whole Home Electrification                         3671.1
High Efficiency Heat Pump with Electric Heat Backup                2803.1
Mininum Efficiency Whole Home Electrification                      2313.8
Enhanced Enclosure                                                 1833.0
Basic Enclosure                                                    1579.6
Mininum Efficiency Heat Pump with Electric Heat Backup             1544.9
Mininum Efficiency Heat Pump with Existing Heat Backup              943.6
Heat Pump Water Heater                                              548.9
Electric Cooking                                                     98.2
Heat Pump Clothes Dryer  

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    4098.2
Basic Enclosure + High Efficiency Whole Home Electrification       4045.1
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     4011.4
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        3958.4
High Efficiency Whole Home Electrification                         3569.1
High Efficiency Heat Pump with Electric Heat Backup                3035.4
Enhanced Enclosure                                                 1884.3
Mininum Efficiency Heat Pump with Electric Heat Backup             1714.9
Mininum Efficiency Heat Pump with Existing Heat Backup             1714.9
Basic Enclosure                                                    1697.0
Mininum Efficiency Whole Home Electrification                       417.0
Heat Pump Water Heater                                              339.8
Electric Cooking                                                     90.7
Heat Pump Clothes Dryer  

## 3.5. Pecent Stock with Positive and Negtive Savings

### Segment 1

In [49]:
segment = 'segment1'
results = figure_positive_negative_savings(segment, income, community_results) 
saving_results[f'segment1_{income}_pct_bill_positive_savings_%'] = results[0]*100
saving_results[f'segment1_{income}_pct_emission_positive_savings_%'] = results[1]*100
saving_results[f'segment1_{income}_pct_energy_positive_savings_%'] = results[2]*100

### Segment 2

In [50]:
segment = 'segment2'
results = figure_positive_negative_savings(segment, income, community_results)
saving_results[f'segment2_{income}_pct_bill_positive_savings_%'] = results[0]*100
saving_results[f'segment2_{income}_pct_emission_positive_savings_%'] = results[1]*100
saving_results[f'segment2_{income}_pct_energy_positive_savings_%'] = results[2]*100

## 3.6. Average Energy Burden (%) Before and after Upgrade

In [51]:
results = segment_mean_energy_burden('post-upgrade_energy_burden_2023_cbills.%', community_results) 
saving_results[[f'segment1_{income}_energy_burden_before_upgrade_%',f'segment1_{income}_energy_burden_after_upgrade_%']] = results[0]
saving_results[[f'segment2_{income}_energy_burden_before_upgrade_%',f'segment2_{income}_energy_burden_after_upgrade_%']] = results[1]

energy burden (%) before and after upgrade
segment1


/var/folders/qt/5nrnlf6148z2jxyxx_5yqgl0z8pq2p/T/ipykernel_79926/2800418672.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/qt/5nrnlf6148z2jxyxx_5yqgl0z8pq2p/T/ipykernel_79926/2800418672.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/qt/5nrnlf6148z2jxyxx_5yqgl0z8pq2p/T/ipykernel_79926/2800418672.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

,before,after
upgrade_name,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,15.6,7.7
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,15.6,7.7
Basic Enclosure + High Efficiency Whole Home Electrification,15.6,7.8
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,15.6,7.8
High Efficiency Whole Home Electrification,15.6,8.8
High Efficiency Heat Pump with Electric Heat Backup,15.6,10.6
Mininum Efficiency Whole Home Electrification,15.6,11.8
Enhanced Enclosure,15.6,12.2
Basic Enclosure,15.6,12.6


segment2


,before,after
upgrade_name,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,21.4,9.8
Basic Enclosure + High Efficiency Whole Home Electrification,21.4,9.9
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,21.4,9.9
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,21.4,10.1
High Efficiency Whole Home Electrification,21.4,11.4
High Efficiency Heat Pump with Electric Heat Backup,21.4,12.7
Enhanced Enclosure,21.4,16.1
Basic Enclosure,21.4,16.5
Mininum Efficiency Heat Pump with Electric Heat Backup,21.4,16.7


/var/folders/qt/5nrnlf6148z2jxyxx_5yqgl0z8pq2p/T/ipykernel_79926/505849424.py:3: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`



In [52]:
saving_results.to_csv(fig_dir / f'saving_results_{community}.csv')

### Save html file

In [53]:
! jupyter nbconvert --output-dir='./results/north_birmingham/' --to=html --no-input saving_results_reports_for_north_birmingham.ipynb

[NbConvertApp] Converting notebook saving_results_reports_for_north_birmingham.ipynb to html
/Users/nmerket/mambaforge/envs/datatools/share/jupyter/nbconvert/templates/base/display_priority.j2:32: UserWarning: Your element with mimetype(s) dict_keys(['application/vnd.plotly.v1+json']) is not able to be represented.
  {%- elif type == 'application/javascript' -%}
[NbConvertApp] Writing 797444 bytes to results/north_birmingham/saving_results_reports_for_north_birmingham.html


In [54]:
rows = ['Basic Enclosure', 'Enhanced Enclosure', 'Heat Pump Water Heater', 'Mininum Efficiency Heat Pump with Electric Heat Backup']
saving_results.loc[rows, ['whole_community_saving_bill_total_usd', 'whole_community_saving_emission_total_kgCO2e']].to_clipboard()

In [55]:
community_results.columns.values

array(['building_id', 'build_existing_model.ahs_region',
       'build_existing_model.area_median_income',
       'build_existing_model.ashrae_iecc_climate_zone_2004_2_a_split',
       'build_existing_model.ashrae_iecc_climate_zone_2004',
       'build_existing_model.bathroom_spot_vent_hour',
       'build_existing_model.bedrooms',
       'build_existing_model.building_america_climate_zone',
       'build_existing_model.cec_climate_zone',
       'build_existing_model.ceiling_fan',
       'build_existing_model.census_division_recs',
       'build_existing_model.census_division',
       'build_existing_model.census_region', 'build_existing_model.city',
       'build_existing_model.clothes_dryer',
       'build_existing_model.clothes_washer_presence',
       'build_existing_model.clothes_washer',
       'build_existing_model.cooking_range',
       'build_existing_model.cooling_setpoint_has_offset',
       'build_existing_model.cooling_setpoint_offset_magnitude',
       'build_existing_mod

In [56]:
community_results['upgrade_name'].unique()

array(['baseline', 'Basic Enclosure', 'Enhanced Enclosure',
       'Mininum Efficiency Heat Pump with Electric Heat Backup',
       'High Efficiency Heat Pump with Electric Heat Backup',
       'Mininum Efficiency Heat Pump with Existing Heat Backup',
       'Basic Enclosure + HPWH + High Efficiency HP/Electric Backup',
       'Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup',
       'Heat Pump Water Heater', 'Electric Clothes Dryer',
       'Heat Pump Clothes Dryer', 'Electric Cooking', 'Induction Cooking',
       'Mininum Efficiency Whole Home Electrification',
       'High Efficiency Whole Home Electrification',
       'Basic Enclosure + High Efficiency Whole Home Electrification',
       'Enhanced Enclosure + High Efficiency Whole Home Electrification'],
      dtype=object)

In [57]:
floor_areas = pd.read_parquet(
    'data_/community_building_samples_with_upgrade_cost_and_bill/north_birmingham/up03__north_birmingham.parquet',
    columns=['building_id', 
             'apply_upgrade.upgrade_name',
             'upgrade_costs.floor_area_conditioned_ft_2', 
             'upgrade_costs.size_heating_system_primary_k_btu_h',
             'completed_status']
)
floor_areas['apply_upgrade.upgrade_name'].unique()

array(['Heat pumps, min-efficiency, electric backup'], dtype=object)

In [58]:
floor_areas.columns.values

array(['building_id', 'apply_upgrade.upgrade_name',
       'upgrade_costs.floor_area_conditioned_ft_2',
       'upgrade_costs.size_heating_system_primary_k_btu_h',
       'completed_status'], dtype=object)

In [59]:
floor_areas

,building_id,apply_upgrade.upgrade_name,upgrade_costs.floor_area_conditioned_ft_2,upgrade_costs.size_heating_system_primary_k_btu_h,completed_status
0,239,"Heat pumps, min-efficiency, electric backup",1690.0,39.22,Success
1,789,"Heat pumps, min-efficiency, electric backup",1220.0,62.34,Success
2,1185,"Heat pumps, min-efficiency, electric backup",853.0,11.65,Success
3,1433,"Heat pumps, min-efficiency, electric backup",2663.0,46.69,Success
4,1907,"Heat pumps, min-efficiency, electric backup",1220.0,43.26,Success
...,...,...,...,...,...
1150,546426,"Heat pumps, min-efficiency, electric backup",1220.0,60.20,Success
1151,548159,"Heat pumps, min-efficiency, electric backup",617.0,12.69,Success
1152,548178,"Heat pumps, min-efficiency, electric backup",2663.0,69.32,Success
1153,548982,"Heat pumps, min-efficiency, electric backup",617.0,18.12,Success


In [60]:
df2 = community_results.merge(floor_areas, how='left', on='building_id')

In [61]:
upg_name = 'High Efficiency Heat Pump with Electric Heat Backup'
df2[df2['upgrade_name'] == upg_name].groupby('segment')[[
    'upgrade_costs.floor_area_conditioned_ft_2',
    'upgrade_costs.size_heating_system_primary_k_btu_h',
]].mean()

,upgrade_costs.floor_area_conditioned_ft_2,upgrade_costs.size_heating_system_primary_k_btu_h
segment,,
nan,1702.605573,43.588006
segment1,1444.388060,50.346642
segment2,1514.806818,52.268068
